In [25]:
import os
from collections import Counter

import meshio
import networkx as nx
import numpy as np
import polyscope as ps
import polyscope.imgui as psim


In [26]:
file_path = os.path.join(os.getcwd(), "../../meshes/plat_tetra.msh")
if not os.path.exists(file_path):
    raise FileNotFoundError(f"Mesh file not found: {file_path}")

In [27]:

# Global transformation parameters
scale_factor = 1.0
rotation_degrees = [0.0, 0.0, 0.0]  # Rotation around X, Y, Z axes in degrees
translation = [0.0, 0.0, 0.0]

# Global variables for mesh and surfaces
original_points = None
original_cells = None
transformed_points = None
boundary_surfaces = []
surface_mesh_names = []
selected_surface = -1

# Colors
DEFAULT_COLOR = [0.8, 0.8, 0.8]
SELECTED_COLOR = [1.0, 0.0, 0.0]

In [28]:

def load_mesh(file_path):
    """Load a mesh file using meshio and return its points and cells."""
    mesh = meshio.read(file_path)
    return mesh.points, mesh.cells

In [29]:
def extract_boundary_faces(cells):
    """Extract boundary faces from tetrahedral or triangular meshes."""
    all_faces = []
    for cell_block in cells:
        if cell_block.type == "tetra":
            for c in cell_block.data:
                faces = [
                    tuple(sorted([c[0], c[1], c[2]])),
                    tuple(sorted([c[0], c[1], c[3]])),
                    tuple(sorted([c[0], c[2], c[3]])),
                    tuple(sorted([c[1], c[2], c[3]])),
                ]
                all_faces.extend(faces)
        elif cell_block.type == "triangle":
            for c in cell_block.data:
                faces = tuple(sorted(c))
                all_faces.append(faces)

    # Count the occurrences of each face
    face_count = Counter(all_faces)
    # Boundary faces are those that appear only once
    boundary_faces = [f for f, count in face_count.items() if count == 1]
    return np.array(boundary_faces)


In [30]:
def group_boundary_faces(boundary_faces):
    """Group boundary faces into connected surfaces using networkx."""
    G = nx.Graph()
    num_faces = len(boundary_faces)
    G.add_nodes_from(range(num_faces))

    # Create a mapping from edges to face indices
    edge_to_faces = {}
    for i, face in enumerate(boundary_faces):
        edges = [
            tuple(sorted([face[0], face[1]])),
            tuple(sorted([face[1], face[2]])),
            tuple(sorted([face[0], face[2]]))
        ]
        for edge in edges:
            if edge not in edge_to_faces:
                edge_to_faces[edge] = []
            edge_to_faces[edge].append(i)

    # Add edges between faces that share a common edge
    for edge, faces in edge_to_faces.items():
        if len(faces) > 1:
            for j in range(len(faces)):
                for k in range(j + 1, len(faces)):
                    G.add_edge(faces[j], faces[k])

    # Identify connected components (surfaces)
    surfaces = list(nx.connected_components(G))
    return surfaces

In [31]:
def apply_transforms(points):
    """Apply scaling, rotation, and translation to the points."""
    p = points * scale_factor
    rx, ry, rz = np.radians(rotation_degrees)
    # Rotation matrices around X, Y, Z
    Rx = np.array([
        [1, 0, 0],
        [0, np.cos(rx), -np.sin(rx)],
        [0, np.sin(rx), np.cos(rx)]
    ])
    Ry = np.array([
        [np.cos(ry), 0, np.sin(ry)],
        [0, 1, 0],
        [-np.sin(ry), 0, np.cos(ry)]
    ])
    Rz = np.array([
        [np.cos(rz), -np.sin(rz), 0],
        [np.sin(rz), np.cos(rz), 0],
        [0, 0, 1]
    ])
    # Combined rotation
    R = Rz @ Ry @ Rx
    p = p @ R.T
    # Translation
    p += translation
    return p

In [32]:
def register_surface_meshes(surfaces, boundary_faces, transformed_points):
    """Register each surface as a separate mesh in Polyscope."""
    global boundary_surfaces, surface_mesh_names
    boundary_surfaces = surfaces
    surface_mesh_names = []

    for i, surface in enumerate(boundary_surfaces):
        # Extract faces for this surface
        surface_faces = boundary_faces[list(surface)]
        # Get unique vertices and remap face indices
        unique_vertices, inverse_indices = np.unique(surface_faces, return_inverse=True)
        surface_points = transformed_points[unique_vertices]
        surface_faces_unique = inverse_indices.reshape(-1, 3)
        # Define mesh name
        mesh_name = f"Surface {i+1}"
        # Register the surface mesh, initially hidden
        ps.register_surface_mesh(mesh_name, surface_points, surface_faces_unique,
                                 color=DEFAULT_COLOR, smooth_shade=True, enabled=False)
        surface_mesh_names.append(mesh_name)


In [33]:
def update_selected_surface():
    """Update the visibility and color of surfaces based on the selected_surface index."""
    for i, mesh_name in enumerate(surface_mesh_names):
        if i == selected_surface:
            ps.get_surface_mesh(mesh_name).set_enabled(True)
            ps.get_surface_mesh(mesh_name).set_color(SELECTED_COLOR)
        else:
            ps.get_surface_mesh(mesh_name).set_enabled(False)
            ps.get_surface_mesh(mesh_name).set_color(DEFAULT_COLOR)

In [34]:
def user_callback():
    """Polyscope UI for user interaction: Surface Selector."""
    global selected_surface

    if boundary_surfaces:
        # Create a list of surface names
        surface_names = [f"Surface {i+1}" for i in range(len(boundary_surfaces))]
        # Current selection index
        current_selection = selected_surface if selected_surface >= 0 else -1

        # Dropdown menu for selecting surfaces
        changed, new_selection = psim.Combo("Select Surface", current_selection, surface_names, 5)
        if changed:
            selected_surface = new_selection
            update_selected_surface()

        # Reset Selection Button
        if psim.Button("Reset Selection"):
            selected_surface = -1
            # Reset all surfaces to default state
            for mesh_name in surface_mesh_names:
                ps.get_surface_mesh(mesh_name).set_enabled(False)
                ps.get_surface_mesh(mesh_name).set_color(DEFAULT_COLOR)
    else:
        psim.Text("No boundary surfaces available for selection.")

In [35]:
def main():
    global original_points, original_cells, transformed_points, boundary_surfaces, surface_mesh_names, selected_surface

    # Load mesh
    original_points, original_cells = load_mesh(file_path)

    # Apply initial transformations
    transformed_points = apply_transforms(original_points)

    # Initialize Polyscope
    ps.init()
    ps.set_navigation_style("turntable")

    # Extract boundary faces
    boundary_faces = extract_boundary_faces(original_cells)

    # Register the original mesh
    ps.register_surface_mesh("Original Mesh", transformed_points, boundary_faces, smooth_shade=True)

    # Group boundary faces into surfaces
    surfaces = group_boundary_faces(boundary_faces)

    # Register each surface as a separate mesh
    register_surface_meshes(surfaces, boundary_faces, transformed_points)

    # Set the user callback for GUI interactions
    ps.set_user_callback(user_callback)

    # Show the Polyscope window
    ps.show()

In [36]:
if __name__ == "__main__":
    main()